In [1]:
import Watchmen

w = Watchmen.Watchmen(openai_api_key_path='E:\\keys\\openAIApiKey.txt', youtube_api_key_path='E:\\keys\\youtubeDataApiKey.txt')
#w.update_new_classifications()
#w.fetch_new_transcripts_for_channels(max_pages=3)
#w.librarian.compile_entity_index()
#w.librarian.compile_topic_index()
#w.librarian.compile_analysis_files()

#print(w.scholar.text)

setting up librarian...
setting up youtube explorer.
setting up scholar...


In [14]:
import pandas as pd
import os
import pyarrow as pa
from pyarrow import parquet as pq

def get_main_topic(video_id):
    if os.path.exists(f'./data/classifications/{video_id}_classification.parquet'):
        dfc = pd.read_parquet(f'./data/classifications/{video_id}_classification.parquet')
        if 'main_topic' in dfc.columns:
            return dfc['main_topic'][0]
        else:
            return None
    else:
        return None

df = pd.read_parquet('./data/library_index.parquet')
df[df['main_topic']>''].count()
#df['main_topic'] = df['source_id'].apply(lambda x: get_main_topic(x))

#table = pa.Table.from_pandas(df)
#pq.write_table(table, './data/library_index.parquet', use_dictionary=True, compression='gzip')

lib_key              900
source_id            900
rec_mod_dtm          900
doc_type_id          900
source_system_id     900
path                 900
summary              900
summary_reasoning    900
sentiment            900
primary_emotion      900
secondary_emotion    900
main_topic           900
dtype: int64

In [21]:
import pandas as pd
import os

dfc = pd.read_csv('./data/youtube_channels_index.csv')
channel_ids = dfc['channel_id'].to_list()

folder = './data/channels/'
all_vids = []

for file in os.listdir(folder):
    dfs = pd.read_parquet(folder + file)
    video_ids = dfs['video_id'].to_list()
    all_vids = all_vids + video_ids

folder = './data/transcripts/'
for file in os.listdir(folder):
    video_id = file.replace('_transcript.parquet','')
    if video_id not in all_vids:
        os.remove(folder + file)

In [43]:
import pandas as pd
import datetime

past_90_days_start = datetime.datetime.now() - datetime.timedelta(days=60)

dft = pd.read_parquet('./data/analysis/topic_analysis_data.parquet')
dft['published_at'] = pd.to_datetime(dft['published_at'], utc=True)
dft['cutoff_date'] = past_90_days_start
dft['cutoff_date'] = pd.to_datetime(dft['cutoff_date'], utc=True)

dft = dft[dft['published_at'] >= dft['cutoff_date']]

dft['published_at_et'] = dft['published_at'].dt.tz_convert('US/Eastern')
dft['published_date_et'] = pd.to_datetime(dft['published_at_et']).dt.date
dft['published_week'] = dft['published_date_et'] - pd.offsets.Week(weekday=6)
dft['published_week'] = pd.to_datetime(dft['published_week']).dt.date
dft.head()

,topic,source_id,title,published_at,channel_title,category,region,rec_mod_dtm,summary,sentiment,primary_emotion,secondary_emotion,cutoff_date,published_at_et,published_date_et,published_week
0,Israeli military operations,-6pq0h1tAFk,Israeli raid forces major Gaza hospital to shu...,2024-12-28 18:27:54+00:00,CBC,News,Canada,2024-12-31 13:06:58,The Israeli military has concluded a raid on t...,False,sadness,sympathy,2024-11-19 14:50:08.797009+00:00,2024-12-28 13:27:54-05:00,2024-12-28,2024-12-22
1,Gaza conflict,-6pq0h1tAFk,Israeli raid forces major Gaza hospital to shu...,2024-12-28 18:27:54+00:00,CBC,News,Canada,2024-12-31 13:06:58,The Israeli military has concluded a raid on t...,False,sadness,sympathy,2024-11-19 14:50:08.797009+00:00,2024-12-28 13:27:54-05:00,2024-12-28,2024-12-22
2,Hamas,-6pq0h1tAFk,Israeli raid forces major Gaza hospital to shu...,2024-12-28 18:27:54+00:00,CBC,News,Canada,2024-12-31 13:06:58,The Israeli military has concluded a raid on t...,False,sadness,sympathy,2024-11-19 14:50:08.797009+00:00,2024-12-28 13:27:54-05:00,2024-12-28,2024-12-22
3,healthcare crisis,-6pq0h1tAFk,Israeli raid forces major Gaza hospital to shu...,2024-12-28 18:27:54+00:00,CBC,News,Canada,2024-12-31 13:06:58,The Israeli military has concluded a raid on t...,False,sadness,sympathy,2024-11-19 14:50:08.797009+00:00,2024-12-28 13:27:54-05:00,2024-12-28,2024-12-22
4,World Health Organization,-6pq0h1tAFk,Israeli raid forces major Gaza hospital to shu...,2024-12-28 18:27:54+00:00,CBC,News,Canada,2024-12-31 13:06:58,The Israeli military has concluded a raid on t...,False,sadness,sympathy,2024-11-19 14:50:08.797009+00:00,2024-12-28 13:27:54-05:00,2024-12-28,2024-12-22


In [25]:
import plotly.express as px

topics_by_channel = dft[dft['topic']!='none'].groupby(['topic','channel_title','primary_emotion','secondary_emotion','published_date_et'])['title'].count().reset_index().rename(columns={'title':'videos','channel_title':'channel'})
topics_by_channel.sort_values(by=['published_date_et','topic','channel'], inplace=True)
topics_by_channel.head()


,topic,channel,primary_emotion,secondary_emotion,published_date_et,videos
2327,companionship,National Post,love,contentment,2015-05-13,1
3671,healing,National Post,love,contentment,2015-05-13,1
4010,imagination,National Post,love,contentment,2015-05-13,1
4481,loneliness,National Post,love,contentment,2015-05-13,1
4854,nostalgia,National Post,love,contentment,2015-05-13,1


In [35]:
import datetime

emotions_over_time = topics_by_channel.groupby(['published_date_et','primary_emotion','channel','topic'])['videos'].sum().reset_index()
emotions_over_time = emotions_over_time[emotions_over_time['published_date_et']>datetime.date(2024,9,1)]
fig = px.line(emotions_over_time, x='published_date_et',y='videos',color='topic', facet_col='channel', facet_row='primary_emotion',height=400)
fig.show()